In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

/var/folders/tl/lnf2sv191t77b2f4hhxqlcx80000gn/T/ipykernel_60271/3709173474.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
chicago_df = pd.read_csv('./chicago_jan_2022_to_december_2023_hourly.csv')
chicago_df.shape
syr_df = pd.read_csv('./syracuse_jan_2022_to_december_2023_hourly.csv')
syr_df.shape
ny_df = pd.read_csv('./ny_jan_2022_to_december_2023_hourly.csv')
ny_df.shape
orlando_df = pd.read_csv('./orlando_jan_2022_to_december_2023_hourly.csv')
orlando_df.shape

(17520, 24)

(17520, 24)

(17520, 24)

(17520, 24)

In [9]:
chicago_df.columns

Index(['name', 'datetime', 'temp', 'feelslike', 'dew', 'humidity', 'precip',
       'precipprob', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
       'icon', 'stations'],
      dtype='object')

In [3]:
merged_weather = pd.concat([chicago_df, syr_df, ny_df, orlando_df], ignore_index=True)
merged_weather.shape

(70080, 24)

In [4]:
# Replacing city names with airport names
dic = {'syracuse': 'SYR', 'new york': 'JFK', 'orlando': 'MCO', 'chicago': 'ORD'}

merged_weather['name'] = merged_weather['name'].apply(lambda row: dic[row.lower()])

merged_weather.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,ORD,2022-01-01T00:00:00,40.90,33.40,36.50,84.14,0.00,0,NaN,0.00,0.00,21.90,13.80,12.00,"1,006.70",100.00,5.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
1,ORD,2022-01-01T01:00:00,39.50,30.80,36.00,86.96,0.00,0,NaN,0.00,0.00,26.20,16.30,15.00,"1,007.10",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
2,ORD,2022-01-01T02:00:00,38.30,31.40,34.70,86.89,0.00,0,NaN,0.00,0.00,NaN,10.40,5.00,"1,007.90",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
3,ORD,2022-01-01T03:00:00,38.50,30.80,34.40,85.10,0.00,0,NaN,0.00,0.00,NaN,12.40,11.00,"1,008.50",100.00,5.60,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
4,ORD,2022-01-01T04:00:00,38.30,29.90,33.50,82.72,0.00,0,NaN,0.00,0.00,26.40,14.10,10.00,"1,008.40",100.00,8.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."


## Required / Selected weather columns: 
- name
- datetime
- temp	
- feelslike
- precip	
- precipprob
- preciptype
- snow
- windgust
- windspeed
- winddir
- severerisk
- cloudcover
- visibility
- conditions

In [6]:
merged_weather.columns
merged_weather_filtered = merged_weather.drop(columns = ['dew','humidity','snowdepth','sealevelpressure', 'solarradiation', 'solarenergy','uvindex','icon','stations'])
merged_weather_filtered.head()

Index(['name', 'datetime', 'temp', 'feelslike', 'dew', 'humidity', 'precip',
       'precipprob', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
       'icon', 'stations'],
      dtype='object')

,name,datetime,temp,feelslike,precip,precipprob,preciptype,snow,windgust,windspeed,winddir,cloudcover,visibility,severerisk,conditions
0,ORD,2022-01-01T00:00:00,40.90,33.40,0.00,0,NaN,0.00,21.90,13.80,12.00,100.00,5.40,NaN,Overcast
1,ORD,2022-01-01T01:00:00,39.50,30.80,0.00,0,NaN,0.00,26.20,16.30,15.00,100.00,3.30,NaN,Overcast
2,ORD,2022-01-01T02:00:00,38.30,31.40,0.00,0,NaN,0.00,NaN,10.40,5.00,100.00,3.30,NaN,Overcast
3,ORD,2022-01-01T03:00:00,38.50,30.80,0.00,0,NaN,0.00,NaN,12.40,11.00,100.00,5.60,NaN,Overcast
4,ORD,2022-01-01T04:00:00,38.30,29.90,0.00,0,NaN,0.00,26.40,14.10,10.00,100.00,8.40,NaN,Overcast


In [8]:
merged_weather_filtered.columns

Index(['name', 'datetime', 'temp', 'feelslike', 'precip', 'precipprob',
       'preciptype', 'snow', 'windgust', 'windspeed', 'winddir', 'cloudcover',
       'visibility', 'severerisk', 'conditions'],
      dtype='object')

In [11]:
merged_weather_filtered.to_csv('./merged_weather_hourly_2022_to_2023_filtered_columns.csv', index=False)